In [ ]:
"""
MNIST データ (MNIST handwritten digit database) と呼ばれる、0-9 の
手書きの数字の画像データ (縦 82 x 横 82 px = 計 784 px) を利用します。
このデータセットは 3 つに分かれており、
訓練用 (Train) データ 55,000 件、
テスト用 (Test) データ 10,000 件、
検証用 (Validation) データ 5,000 件
の 計 70,000 件で構成されています。

以下コードで MNIST データを読み込みます。
TensorFlow には、クイックに検証する目的で、MNIST データをダウンロードし、
読み込む機能があらかじめ組み込まれています。

"""
# データの読み込み
DIR_DATA = "/var/tmp/work/MNIST_data"

# MNIST データセットを読み込む
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(DIR_DATA, one_hot=True)

"""
今回はSoftmax 回帰による予測モデルを作成します。
Softmax 回帰は、多変量のロジスティック回帰を利用した多値分類の
アルゴリズムで、予測モデルは以下式で表せます。
　　 y = Softmax(Wx + b)
   
Y が目的変数 (0-9の数字)、
X が入力変数 (256 階調のグレースケールで示される、
  各ピクセルの濃淡を 0.0-1.0 で表したもの)、
W が重み、
b がバイアスの行列を意味

入力データと出力データの格納領域として、プレースホルダ (placeholder) を作成します。
また、重み行列とバイアス行列の格納領域として、Variable を作成します。

今回は、最適な重みとバイアスを推定するためのコスト関数として、
交差エントロピー (cross entropy) と呼ばれる指標を用います。
この交差エントロピーが最も小さくなるように、TensorFlow に 
GradientDescentOptimizer として実装されている、
勾配降下法 (Gradient descent) を用いて学習を進めます。

"""

In [ ]:
# Tensorflow を読み込み
import tensorflow as tf

plot_y = []

# 予測精度の計算と記憶
# y : 実際の値  　y_: 予測値  　ecpoch_i : 試行回数インデックス
def calc_accurary(y, y_, epoch_i):
    # 実際の値と予測された値が同じであるか確認
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
        
    # 平均値を求め、予測精度を求める
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    # x にテストデータ、y_ に実際の値をあてはめ、上記で作成した式を実行
    train_feed = {x: mnist.test.images, y_: mnist.test.labels}
    val = sess.run(accuracy, feed_dict=train_feed)
    plot_y.append(val)
    if epoch_i % 10 == 0:
        print('epoch:[{:>2d}] accurary:[{:>2.8f}]'.format(epoch_i, val))


In [ ]:
# 入力データ格納用の 784 px 分のプレースホルダを作成
x = tf.placeholder(tf.float32, [None, 784])
# 重み (784 x 10 の行列) の Variable を定義
W = tf.Variable(tf.zeros([784, 10]))
# バイアス (長さ 10 の行列) の Variable を定義
b = tf.Variable(tf.zeros([10]))
# エポック　試行回数
epoch = 1500
# 学習率
learning_rate = 0.05

# ソフトマックス回帰による予測式を定義
y = tf.nn.softmax(tf.matmul(x, W) + b)
# 出力データ (予測値) 格納用のプレースホルダ
y_ = tf.placeholder(tf.float32, [None, 10])

# 交差エントロピーを最小化するよう、学習を行う式を以下のように定義
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

# TensorFlow session
with tf.Session() as sess:
    # you need to initialize all variables
    tf.global_variables_initializer().run()

    for i in range(epoch + 1):
        # 訓練用データから 100 件取得
        batch_xs, batch_ys = mnist.train.next_batch(100)
        # train_step を実行
        train_feed = {x: batch_xs, y_: batch_ys}
        sess.run(train_step, feed_dict=train_feed)
        
        calc_accurary(y, y_, i)



In [ ]:
# 学習過程のグラフ表示
import plotly
plotly.offline.init_notebook_mode(connected=False)
from plotly.graph_objs import Scatter, Layout

def plot(plot_dic, height=500, width=500, **kwargs):
    kwargs['output_type'] = 'div'
    plot_str = plotly.offline.plot(plot_dic, **kwargs)
    print('%%angular <div style="height: %ipx; width: %spx"> %s </div>' % (height, width, plot_str))

plot_x = np.arange(epoch + 1)
plot({
    "data": [
        Scatter(x=plot_x, y=plot_y, name="accuracy", yaxis="y2")
    ],
    "layout": Layout(
        title="tensorflow learning process"
    )
})
